In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
df=pd.read_csv('Travel.csv')
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null   float64
 14  Passport

## data cleaning

In [9]:
df.isnull().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [11]:
df['Gender'].value_counts() ## there is only two gender in the world

Gender
Male       2916
Female     1817
Fe Male     155
Name: count, dtype: int64

In [12]:
df['Gender']=df['Gender'].replace('Fe Male','Female')

In [13]:
df['Gender'].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [14]:
df['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [15]:
## as single and unmarried are same 
df['MaritalStatus']=df['MaritalStatus'].replace('Single','Unmarried')

In [16]:
df['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Unmarried    1598
Divorced      950
Name: count, dtype: int64

In [17]:
### check the missing values
## these values are feature with nan values
features_with_nan=[features for features in df.columns if df[features].isnull().sum()>=1]
for feature in features_with_nan:
    print(feature,np.round(df[feature].isnull().mean()*100,5),'% missing value')

Age 4.62357 % missing value
TypeofContact 0.51146 % missing value
DurationOfPitch 5.13502 % missing value
NumberOfFollowups 0.92062 % missing value
PreferredPropertyStar 0.53191 % missing value
NumberOfTrips 2.86416 % missing value
NumberOfChildrenVisiting 1.35025 % missing value
MonthlyIncome 4.76678 % missing value


In [18]:
df[features_with_nan].select_dtypes(exclude='object').describe()

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


In [20]:
## age
df.Age.fillna(df.Age.median(),inplace=True)
## typeofcontact
df.TypeofContact.fillna(df.TypeofContact.mode()[0],inplace=True)
## duration of pitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(),inplace=True)
## number of follow ups
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0],inplace=True)
## preferred property star
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0],inplace=True)
### number of trips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(),inplace=True)
## number of children visitiing
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0],inplace=True)
### monthly income
df.MonthlyIncome.fillna(df.MonthlyIncome.median(),inplace=True)

In [21]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [22]:
df.drop('CustomerID',inplace=True,axis=1)

### Feature Engineering

In [26]:
df['TotalVisiting']=df['NumberOfChildrenVisiting']+df['NumberOfPersonVisiting']
df.drop(columns=['NumberOfChildrenVisiting','NumberOfPersonVisiting'],axis=1,inplace=True)

In [28]:
## get all the numeric features
num_features=[feature for feature in df.columns if df[feature].dtype!='O']
print('Number of numerical features are: ',len(num_features))

Number of numerical features are:  12


In [30]:
## categorical features
cat_feature=[feature for feature in df.columns if df[feature].dtype=='O']
print('Number of categorical features are ',len(cat_feature))

Number of categorical features are  6


In [31]:
## discrete feature
discrete_feature=[feature for feature in num_features if len(df[feature].unique())<=25]
print('Number of discrete features are ',len(discrete_feature))

Number of discrete features are  9


In [32]:
## continuous feature
continuous_feature=[feature for feature in num_features if feature not in discrete_feature]
print('Number of continuous feature are ',len(continuous_feature))

Number of continuous feature are  3


### Train Test Split


In [33]:
from sklearn.model_selection import train_test_split
x=df.drop(['ProdTaken'],axis=1)
y=df['ProdTaken']

In [34]:
y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [35]:
x.head()

,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisiting
0,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [36]:
## train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((3910, 17), (978, 17))

In [37]:
## create column transformer with 3 types of transformers
cat_feature=x.select_dtypes(include='object').columns
num_features=x.select_dtypes(exclude='object').columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer=StandardScaler()
oh_transformer=OneHotEncoder(drop='first')

preprocessor=ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,cat_feature),
        ('StandardScaler',num_transformer,num_features)
    ]
)



In [38]:
x_train=preprocessor.fit_transform(x_train)
x_test=preprocessor.transform(x_test)

### Random forest classifier technique


In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score,roc_curve


In [43]:
models={
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier()
}
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train) # train model

    ## make predictions
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    ## training set performance
    model_training_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred,average='weighted')
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_recauc_score=roc_auc_score(y_train,y_train_pred)

    ## test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred,average='weighted')
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_recauc_score=roc_auc_score(y_test,y_test_pred)

    print(list(models.keys())[i])

    print('Model performance for training set')
    print('accuracy: {:.4f}'.format(model_training_accuracy))
    print('f1 score: {:.4f}'.format(model_train_f1))
    print('precision: {:.4f}'.format(model_train_precision))
    print('recall: {:.4f}'.format(model_train_recall))
    print('roc auc score: {:.4f}'.format(model_train_recauc_score))

    print('----------------------------------------------')
    print('Model performance for test data')
    print('Model performance for training set')
    print('accuracy: {:.4f}'.format(model_test_accuracy))
    print('f1 score: {:.4f}'.format(model_test_f1))
    print('precision: {:.4f}'.format(model_test_precision))
    print('recall: {:.4f}'.format(model_test_recall))
    print('roc auc score: {:.4f}'.format(model_test_recauc_score))

    print('='*35)
    print('\n')



Random Forest
Model performance for training set
accuracy: 1.0000
f1 score: 1.0000
precision: 1.0000
recall: 1.0000
roc auc score: 1.0000
----------------------------------------------
Model performance for test data
Model performance for training set
accuracy: 0.9274
f1 score: 0.9219
precision: 0.9615
recall: 0.6545
roc auc score: 0.8240


Decision Tree
Model performance for training set
accuracy: 1.0000
f1 score: 1.0000
precision: 1.0000
recall: 1.0000
roc auc score: 1.0000
----------------------------------------------
Model performance for test data
Model performance for training set
accuracy: 0.9243
f1 score: 0.9239
precision: 0.8162
recall: 0.7906
roc auc score: 0.8737


